In [1]:
import pandas as pd
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import tensorflow as tf
from keras import layers
from keras.models import Sequential

EPOCHS= 30
act= "swish"
opt= tf.keras.optimizers.Adam(learning_rate=0.001)
from sklearn.model_selection import train_test_split
import numpy as np
from gensim.models import FastText
import tensorflow as tf
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Layer
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Embedding, Input, Dense, LSTM, Dropout, Flatten, Bidirectional, Conv1D, concatenate, SpatialDropout1D
from tensorflow.keras.layers import MaxPooling1D, MaxPooling2D, GlobalMaxPooling1D, GlobalAveragePooling2D, GlobalAveragePooling1D 
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler, EarlyStopping

In [2]:
path = "/content/drive/MyDrive/Datasets/g10words-10authors.csv"

In [3]:
df = pd.read_csv(path)

In [4]:
df.USER.value_counts()

RRPSpeaks          2821
kolhe_amol         2694
PawarSpeaks        2481
rajeshtope11       2313
samant_uday        1779
sangrambhaiya      1741
Subhash_Desai      1669
wankhedeprafull    1614
uddhavthackeray    1578
BodkheShilpa       1388
Name: USER, dtype: int64

In [5]:
df.word_count.unique()

array([13, 14, 12, 18, 21, 20, 16, 31, 19, 30, 28, 11, 23, 15, 22, 24, 40,
       17, 36, 27, 34, 37, 25, 39, 29, 32, 35, 26, 41, 46, 38, 42, 33, 45,
       44, 43, 51, 49, 47, 50, 48, 52])

In [6]:
Y_train = df['USER']
Y_train.head()

0    BodkheShilpa
1    BodkheShilpa
2    BodkheShilpa
3    BodkheShilpa
4    BodkheShilpa
Name: USER, dtype: object

In [7]:
df['label'] = le.fit_transform(Y_train)

In [8]:
df[['USER', 'label']].drop_duplicates(keep='first')

USER  label
0         BodkheShilpa      0
1388        kolhe_amol      4
4082       PawarSpeaks      1
6563      rajeshtope11      5
8876         RRPSpeaks      2
11697      samant_uday      6
13476    sangrambhaiya      7
15217    Subhash_Desai      3
16886  uddhavthackeray      8
18464  wankhedeprafull      9

In [9]:
X_train = df['clean_text']
Y_train = df['label']

In [10]:
sentences_ted = []

In [11]:
X_train = X_train.astype(str)

In [12]:
for sent_str in X_train:
  tokens = sent_str.lower().split()
  sentences_ted.append(tokens)

In [13]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
vocab_size = len(tokenizer.word_index) + 1

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

In [15]:
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [16]:
vocab_size

69941

In [17]:
lens_train = [len(i) for i in X_train]
lens_test = [len(i) for i in X_test]
lens = lens_train + lens_test

maxlen = np.max(lens)
print('Max len:', maxlen)

Max len: 53


In [18]:
s = pd.Series(lens)
s.value_counts()

34    895
36    888
33    864
37    822
35    816
32    779
31    676
30    669
29    667
15    642
38    637
28    591
16    582
25    573
23    571
14    570
22    558
12    553
39    547
27    543
13    531
24    521
26    520
17    518
18    514
41    506
11    506
40    497
19    480
20    456
21    454
43    376
42    283
44    158
45    107
46     70
47     51
48     32
49     22
50     18
51     10
52      3
10      1
53      1
dtype: int64

In [19]:
X_train = pad_sequences(X_train, padding='post', maxlen=45)
X_test = pad_sequences(X_test, padding='post', maxlen=45)

In [20]:
model_ted = FastText(sentences_ted, size=300, window=5, min_count=5, workers=8,sg=1)

In [21]:
word_index = tokenizer.word_index
embedding_matrix = np.zeros((vocab_size, 300))
for word,i in word_index.items():
    try:
        embedding_vector = model_ted.wv[word]
    except:
        print(word, 'not found')
        embedding_vector = None
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

91 not found
०१ not found
०९ not found
०८ not found
66 not found
९२ not found
८६ not found
९७ not found
४९ not found
एफ not found
सढळ not found
84084 not found
ज not found
र not found
५८ not found
97 not found
81 not found
५९ not found
71 not found
85 not found
90 not found
60 not found
८४ not found
८५ not found
०७ not found
४६ not found
भ not found
01 not found
47 not found
े not found
० not found
49 not found
अढळ not found
थॉमस not found
आए। not found
ऑइल not found
४३ not found
32 not found
34 not found
360 not found
43 not found
07 not found
92 not found
73 not found
76 not found
62 not found
09 not found
८८ not found
9249 not found
9496 not found
९४ not found
हब not found
त not found
मठ not found
बढो not found
ईसुफ not found
ह not found
गय not found
यह not found
छळ not found
ढीग not found
ऐ not found
लुई not found
उ not found
इद not found
ी not found
ऊब not found
68 not found
74 not found
38 not found
94 not found
82 not found
77 not found
042 not found
46 not found
03 not found
94

In [22]:
myCNN = Sequential()
myCNN.add(Embedding(input_dim=vocab_size, output_dim=300, weights=[embedding_matrix], input_length=45))
myCNN.add(Conv1D(filters = 32 , kernel_size=3,activation='relu'))
myCNN.add(GlobalMaxPooling1D())
myCNN.add(Dropout(0.3))
myCNN.add(Dense(10, activation='softmax'))
myCNN.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['acc'])

In [23]:
myCNN.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 45, 300)           20982300  
                                                                 
 conv1d (Conv1D)             (None, 43, 32)            28832     
                                                                 
 global_max_pooling1d (Globa  (None, 32)               0         
 lMaxPooling1D)                                                  
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 10)                330       
                                                                 
Total params: 21,011,462
Trainable params: 21,011,462
Non-trainable params: 0
____________________________________________

In [24]:
checkpoint_filepath = './Checkpoint/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
ec = EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)
reduce_lr_callback = ReduceLROnPlateau( monitor='val_acc', 
                                       factor=0.1, 
                                       patience=2, 
                                       min_lr=0.0005, 
                                       verbose=2)

In [25]:
history = myCNN.fit(X_train, y_train, epochs=10, verbose=1, validation_data=(X_test, y_test), batch_size=128, callbacks=[model_checkpoint_callback,ec,reduce_lr_callback]) 
loss, accuracy = myCNN.evaluate(X_train, y_train, verbose=1)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = myCNN.evaluate(X_test, y_test, verbose=1)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Epoch 1/10
126/126 [==============================] - 14s 20ms/step - loss: 1.8543 - acc: 0.3484 - val_loss: 1.5151 - val_acc: 0.5374 - lr: 0.0010
Epoch 2/10
126/126 [==============================] - 3s 20ms/step - loss: 1.3797 - acc: 0.5500 - val_loss: 1.2510 - val_acc: 0.6225 - lr: 0.0010
Epoch 3/10
126/126 [==============================] - 2s 15ms/step - loss: 1.0507 - acc: 0.6728 - val_loss: 1.0928 - val_acc: 0.6643 - lr: 0.0010
Epoch 4/10
126/126 [==============================] - 2s 15ms/step - loss: 0.7769 - acc: 0.7719 - val_loss: 0.9792 - val_acc: 0.6897 - lr: 0.0010
Epoch 5/10
126/126 [==============================] - 2s 15ms/step - loss: 0.5770 - acc: 0.8411 - val_loss: 0.9127 - val_acc: 0.7099 - lr: 0.0010
Epoch 6/10
126/126 [==============================] - 2s 15ms/step - loss: 0.4282 - acc: 0.8843 - val_loss: 0.8786 - val_acc: 0.7201 - lr: 0.0010
Epoch 7/10
126/126 [==============================] - 2s 15ms/step - loss: 0.3208 - acc: 0.9184 - val_loss: 0.8604 - val_ac

In [26]:
myLSTM = Sequential()
myLSTM.add(Embedding(input_dim=vocab_size, output_dim=300, weights=[embedding_matrix], input_length=45))
myLSTM.add(layers.Bidirectional(layers.LSTM(128, recurrent_dropout=0.3,return_sequences=True)))
myLSTM.add(GlobalMaxPooling1D())
myLSTM.add(Dense(64, activation='relu'))
myLSTM.add(Dense(10, activation='softmax'))
myLSTM.compile(optimizer='adam', loss="sparse_categorical_crossentropy", metrics=['acc'])
myLSTM.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 45, 300)           20982300  
                                                                 
 bidirectional (Bidirectiona  (None, 45, 256)          439296    
 l)                                                              
                                                                 
 global_max_pooling1d_1 (Glo  (None, 256)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_1 (Dense)             (None, 64)                16448     
                                                                 
 dense_2 (Dense)             (None, 10)                650       
                                                                 
Total params: 21,438,694
Trainable params: 21,438,694


In [29]:
history = myLSTM.fit(X_train, y_train, epochs=10, verbose=1, validation_data=(X_test, y_test), batch_size=128, callbacks=[model_checkpoint_callback,ec,reduce_lr_callback]) 

Epoch 1/10
126/126 [==============================] - 44s 342ms/step - loss: 0.0064 - acc: 0.9979 - val_loss: 1.4329 - val_acc: 0.7390 - lr: 0.0010
Epoch 2/10
126/126 [==============================] - 42s 331ms/step - loss: 0.0079 - acc: 0.9975 - val_loss: 1.3501 - val_acc: 0.7413 - lr: 0.0010
Epoch 3/10
126/126 [==============================] - 42s 337ms/step - loss: 0.0105 - acc: 0.9969 - val_loss: 1.2602 - val_acc: 0.7435 - lr: 0.0010
Epoch 4/10
126/126 [==============================] - 41s 323ms/step - loss: 0.0050 - acc: 0.9979 - val_loss: 1.4930 - val_acc: 0.7356 - lr: 0.0010
Epoch 5/10
126/126 [==============================] - 43s 343ms/step - loss: 0.0049 - acc: 0.9985 - val_loss: 1.4410 - val_acc: 0.7532 - lr: 0.0010
Epoch 6/10
126/126 [==============================] - 40s 321ms/step - loss: 0.0036 - acc: 0.9988 - val_loss: 1.4435 - val_acc: 0.7537 - lr: 0.0010


In [31]:
loss, accuracy = myLSTM.evaluate(X_train, y_train, verbose=1)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = myLSTM.evaluate(X_test, y_test, verbose=1)
print("Testing Accuracy:  {:.4f}".format(accuracy))

502/502 [==============================] - 14s 28ms/step - loss: 0.0022 - acc: 0.9991
Training Accuracy: 0.9991
126/126 [==============================] - 4s 28ms/step - loss: 1.4435 - acc: 0.7537
Testing Accuracy:  0.7537


In [32]:
class AttentionLayer(Layer):
    
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)
        
        
    def build(self, input_shape):
        self.W_in = self.add_weight(shape=(input_shape[-1], 1),
                                    initializer='glorot_normal',
                                    trainable=True,
                                    name='input_attention_weights')
       
        self.b = self.add_weight(shape=(input_shape[1], 1),
                                        initializer='zeros',
                                        trainable=True,
                                        name='attention_bias')
        
        super(AttentionLayer, self).build(input_shape)
        
        
    def call(self, lstm_output):
        et = K.squeeze(K.tanh(K.dot(lstm_output, self.W_in) +
                              self.b), axis=-1)
        at = K.expand_dims(K.softmax(et), axis=-1)
        attention_output = lstm_output * at
        return attention_output
    
    
    def compute_output_shape(self, input_state):
        return (input_state[0], input_state[-1])
    
    
    def get_config(self):
        return super(AttentionLayer, self).get_config()

In [33]:
Input1 = Input(shape=(45,))
Embedding_layer1 = Embedding(input_dim=vocab_size,output_dim=300,weights=[embedding_matrix])(Input1)
LSTM1 = Bidirectional(LSTM(64, dropout=0.3, return_sequences=True))(Embedding_layer1)
Attention = AttentionLayer()(LSTM1)
max_pool = MaxPooling1D(pool_size=5)(Attention)
Dropout1 = Dropout(0.3)(max_pool)
Flat = Flatten()(Dropout1)
Dense1 = Dense(512, activation='relu')(Flat)
Dropout2 = Dropout(0.3)(Dense1)
Dense2 = Dense(64, activation='relu')(Dropout2)
Dropout3 = Dropout(0.25)(Dense2)
outputs = Dense(10, activation='softmax')(Dropout3)

model = Model(inputs=Input1, outputs=outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 45)]              0         
                                                                 
 embedding_2 (Embedding)     (None, 45, 300)           20982300  
                                                                 
 bidirectional_1 (Bidirectio  (None, 45, 128)          186880    
 nal)                                                            
                                                                 
 attention_layer (AttentionL  (None, 45, 128)          173       
 ayer)                                                           
                                                                 
 max_pooling1d (MaxPooling1D  (None, 9, 128)           0         
 )                                                               
                                                             

In [40]:
history = model.fit(X_train, y_train, epochs=10, verbose=1, validation_data=(X_test, y_test), batch_size=128, callbacks=[model_checkpoint_callback,ec,reduce_lr_callback]) 
loss, accuracy = model.evaluate(X_train, y_train, verbose=1)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Epoch 1/10
126/126 [==============================] - 3s 27ms/step - loss: 0.0059 - acc: 0.9977 - val_loss: 2.0801 - val_acc: 0.7179 - lr: 0.0010
Epoch 2/10
126/126 [==============================] - 3s 27ms/step - loss: 0.0043 - acc: 0.9985 - val_loss: 2.3934 - val_acc: 0.6920 - lr: 0.0010
Epoch 3/10
125/126 [============================>.] - ETA: 0s - loss: 0.0091 - acc: 0.9970WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0005.
126/126 [==============================] - 3s 26ms/step - loss: 0.0092 - acc: 0.9969 - val_loss: 2.2986 - val_acc: 0.6860 - lr: 0.0010
Epoch 4/10
126/126 [==============================] - 3s 27ms/step - loss: 0.0053 - acc: 0.9982 - val_loss: 2.0515 - val_acc: 0.7159 - lr: 5.0000e-04
Epoch 5/10
126/126 [==============================] - 3s 27ms/step - loss: 0.0031 - acc: 0.9986 - val_loss: 2.1406 - val_acc: 0.7226 - lr: 5.0000e-04
Epoch 6/10
126/126 [==============

In [41]:
Input1 = Input(shape=(45,))
Embedding_layer1 = Embedding(input_dim=vocab_size,output_dim=300,weights=[embedding_matrix])(Input1)
LSTM1 = Bidirectional(LSTM(64, dropout=0.25, return_sequences=True))(Embedding_layer1)
Conv1 = Conv1D(64, kernel_size=5, activation='relu')(LSTM1)
max_pooling1 = MaxPooling1D(pool_size=3)(Conv1)
Dropout2 = Dropout(0.25)(max_pooling1)
Flatten1 = Flatten()(Dropout2)
Dense1 = Dense(512, activation='relu', kernel_regularizer=l2(0.01))(Flatten1)
Dense2 = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(Dense1)
output = Dense(10, activation='softmax')(Dense2)

myBiLSTM_CNN = Model(inputs=Input1, outputs=output)
myBiLSTM_CNN.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
myBiLSTM_CNN.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 45)]              0         
                                                                 
 embedding_3 (Embedding)     (None, 45, 300)           20982300  
                                                                 
 bidirectional_2 (Bidirectio  (None, 45, 128)          186880    
 nal)                                                            
                                                                 
 conv1d_1 (Conv1D)           (None, 41, 64)            41024     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 13, 64)           0         
 1D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 13, 64)            0   

In [43]:
history = myBiLSTM_CNN.fit(X_train, y_train, epochs=10, verbose=1, validation_data=(X_test, y_test), batch_size=128, callbacks=[model_checkpoint_callback,ec,reduce_lr_callback]) 
loss, accuracy = myBiLSTM_CNN.evaluate(X_train, y_train, verbose=1)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = myBiLSTM_CNN.evaluate(X_test, y_test, verbose=1)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Epoch 1/10
126/126 [==============================] - 3s 23ms/step - loss: 0.1328 - acc: 0.9795 - val_loss: 1.2115 - val_acc: 0.7331 - lr: 0.0010
Epoch 2/10
126/126 [==============================] - 3s 23ms/step - loss: 0.1112 - acc: 0.9829 - val_loss: 1.3394 - val_acc: 0.7139 - lr: 0.0010
Epoch 3/10
126/126 [==============================] - ETA: 0s - loss: 0.0944 - acc: 0.9860WARNING:tensorflow:Can save best model only with val_accuracy available, skipping.

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0005.
126/126 [==============================] - 3s 23ms/step - loss: 0.0944 - acc: 0.9860 - val_loss: 1.4200 - val_acc: 0.7258 - lr: 0.0010
Epoch 4/10
502/502 [==============================] - 2s 5ms/step - loss: 0.0454 - acc: 0.9974
Training Accuracy: 0.9974
126/126 [==============================] - 1s 5ms/step - loss: 1.2117 - acc: 0.7425
Testing Accuracy:  0.7425


In [44]:
Input1 = Input(shape=(45,))
Embedding_layer1 = Embedding(input_dim=vocab_size,output_dim=300,weights=[embedding_matrix])(Input1)

LSTM1 = Bidirectional(LSTM(128, dropout=0.25, return_sequences=True))(Embedding_layer1)
Attention = AttentionLayer()(LSTM1)
Conv1 = Conv1D(64, kernel_size=5, activation='relu', padding='same')(Attention)
max_pool = GlobalMaxPooling1D()(Conv1)
Dropout2 = Dropout(0.25)(max_pool)
output = Dense(10, activation='softmax')(Dropout2)

myBiLSTM_CNN_Att = Model(inputs=Input1, outputs=output)
myBiLSTM_CNN_Att.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
myBiLSTM_CNN_Att.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 45)]              0         
                                                                 
 embedding_4 (Embedding)     (None, 45, 300)           20982300  
                                                                 
 bidirectional_3 (Bidirectio  (None, 45, 256)          439296    
 nal)                                                            
                                                                 
 attention_layer_1 (Attentio  (None, 45, 256)          301       
 nLayer)                                                         
                                                                 
 conv1d_2 (Conv1D)           (None, 45, 64)            81984     
                                                                 
 global_max_pooling1d_2 (Glo  (None, 64)               0   

In [46]:
history = myBiLSTM_CNN_Att.fit(X_train, y_train, epochs=10, verbose=1, validation_data=(X_test, y_test), batch_size=128, callbacks=[model_checkpoint_callback,ec,reduce_lr_callback]) 
loss, accuracy = myBiLSTM_CNN_Att.evaluate(X_train, y_train, verbose=1)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = myBiLSTM_CNN_Att.evaluate(X_test, y_test, verbose=1)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Epoch 1/10
126/126 [==============================] - 3s 27ms/step - loss: 0.0495 - acc: 0.9873 - val_loss: 1.1462 - val_acc: 0.7495 - lr: 5.0000e-04
Epoch 2/10
126/126 [==============================] - 3s 26ms/step - loss: 0.0312 - acc: 0.9927 - val_loss: 1.2459 - val_acc: 0.7363 - lr: 5.0000e-04
Epoch 3/10
126/126 [==============================] - 3s 26ms/step - loss: 0.0247 - acc: 0.9933 - val_loss: 1.2270 - val_acc: 0.7490 - lr: 5.0000e-04
Epoch 4/10
502/502 [==============================] - 2s 5ms/step - loss: 0.0085 - acc: 0.9979
Training Accuracy: 0.9979
126/126 [==============================] - 1s 5ms/step - loss: 1.2774 - acc: 0.7493
Testing Accuracy:  0.7493


In [47]:
class AttentionLayer(Layer):
    
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)
        
        
    def build(self, attention_input):
        super(AttentionLayer, self).build(attention_input)
        lstm_shape, cnn_shape = attention_input
        # Attention Weights for LSTM
        self.W_in = self.add_weight(shape=(lstm_shape[-1], 1),
                                    initializer='glorot_normal',
                                    trainable=True,
                                    name='input_attention_weights')
        
        # Attention Weights for CNN
        self.W_context = self.add_weight(shape=(cnn_shape[-1], 1),
                                         initializer='glorot_normal',
                                         trainable=True,
                                         name='context_attention_weights')
                                      
        # Attention Bias
        self.b = self.add_weight(shape=(lstm_shape[1], 1),
                                 initializer='glorot_normal',
                                 trainable=True,
                                 name='attention_bias')
        
        
        
    def call(self, attention_input):
        lstm_output, cnn_output = attention_input
        et = K.squeeze(K.tanh(K.dot(lstm_output, self.W_in) + 
                              K.dot(cnn_output, self.W_context) +
                              self.b), axis=-1)
        at = K.expand_dims(K.softmax(et), axis=-1)
        attention_output = at * lstm_output
        attention_output = K.sum(attention_output, axis=1)
        return attention_output
    
    def get_config(self):
        return super(AttentionLayer, self).get_config()

In [48]:
Input1 = Input(shape=(45,))
Embedding_layer1 = Embedding(input_dim=vocab_size,output_dim=300,weights=[embedding_matrix])(Input1)

# Input 1(BiLSTM):
Lstm = Bidirectional(LSTM(128, dropout=0.3, return_sequences=True))(Embedding_layer1)

# Input 2 (CNN):
Conv1 = Conv1D(filters=256, kernel_size=3, activation='relu', padding='same')(Embedding_layer1)
Dropout1 = Dropout(0.3)(Conv1)
Conv2 = Conv1D(filters=256, kernel_size=4, activation='relu', padding='same')(Embedding_layer1)
Dropout2 = Dropout(0.3)(Conv2)
Conv3 = Conv1D(filters=256, kernel_size=5, activation='relu', padding='same')(Embedding_layer1)
Dropout3 = Dropout(0.3)(Conv3)
merged = concatenate([Dropout1, Dropout2, Dropout3], axis=1)
max_pool = MaxPooling1D(pool_size=3)(merged)

attention = AttentionLayer()([Lstm, max_pool])
Dense1 = Dense(64, activation='relu')(attention)
Dropout_dense = Dropout(0.25)(Dense1)
outputs = Dense(10, activation='softmax')(Dropout_dense)

classifier = Model(inputs=Input1, outputs=outputs)
classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
classifier.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 45)]         0           []                               
                                                                                                  
 embedding_5 (Embedding)        (None, 45, 300)      20982300    ['input_4[0][0]']                
                                                                                                  
 conv1d_3 (Conv1D)              (None, 45, 256)      230656      ['embedding_5[0][0]']            
                                                                                                  
 conv1d_4 (Conv1D)              (None, 45, 256)      307456      ['embedding_5[0][0]']            
                                                                                            

In [49]:
history = classifier.fit(X_train, y_train, epochs=16, verbose=1, validation_data=(X_test, y_test), batch_size=128, callbacks=[model_checkpoint_callback,ec,reduce_lr_callback]) 
loss, accuracy = classifier.evaluate(X_train, y_train, verbose=1)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = classifier.evaluate(X_test, y_test, verbose=1)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Epoch 1/16
126/126 [==============================] - 18s 63ms/step - loss: 1.5774 - acc: 0.4391 - val_loss: 1.2348 - val_acc: 0.5745 - lr: 0.0010
Epoch 2/16
126/126 [==============================] - 5s 39ms/step - loss: 0.9426 - acc: 0.6871 - val_loss: 0.9512 - val_acc: 0.6805 - lr: 0.0010
Epoch 3/16
126/126 [==============================] - 5s 40ms/step - loss: 0.5205 - acc: 0.8338 - val_loss: 0.8196 - val_acc: 0.7385 - lr: 0.0010
Epoch 4/16
126/126 [==============================] - 5s 40ms/step - loss: 0.2835 - acc: 0.9127 - val_loss: 0.7977 - val_acc: 0.7647 - lr: 0.0010
Epoch 5/16
126/126 [==============================] - 5s 40ms/step - loss: 0.1536 - acc: 0.9534 - val_loss: 0.8695 - val_acc: 0.7597 - lr: 0.0010
Epoch 6/16
126/126 [==============================] - 5s 40ms/step - loss: 0.0897 - acc: 0.9735 - val_loss: 0.9631 - val_acc: 0.7674 - lr: 0.0010
Epoch 7/16
502/502 [==============================] - 3s 6ms/step - loss: 0.0218 - acc: 0.9939
Training Accuracy: 0.9939
12